In [1]:
from probabilityPlus import *
from prettytable import *
from utils import print_table
import itertools

In [157]:
#em que  𝑅  é uma Rede de Bayes,  𝑋  e  𝑌  são duas variáveis diferentes na rede  𝑅 , e  𝐸  é um conjunto, 
#possivelmente vazio, de variáveis da Rede de Bayes  𝑅  que não contem nem  𝑋  nem  𝑌 . 
#Essa função verifica se  𝑋  é condicionalmente independente de  𝑌  dado o conjunto de variáveis  𝐸 , 
#devolvendo um valor booleano.
def cond_indep(X, Y, E, R):
    l = []
    path(l, X, Y, R, [])
    if len(l) == 0:
        return False
    for p in l:
        if state(p, E, R):
            return False
    return True

def state(p, E, R): #True if active, False otherwise
    l = p[:3]
    t = p[1:]
    if(len(l) < 3):
        return True
    if inactive(l, E, R): #True if inactive, False otherwise
        return False
    elif len(t) >= 3:
        state(t, E, R)
    else:
        return True
    
def inactive(l, E, R):
    if efeitoCausal(l, E, R):
        if l[1] in E:
            return True
        else:
            return False
    elif efeitoComum(l, E, R):
        if l[1] in E:
            return False
        for node in R.variable_node(l[1]).children:
            if node in E: # check if any descendants of middle cenas is in evidence
                return False
    else:
        if l[1] in E:
            return True

def efeitoCausal(l, E, R): #True if it is
    return (l[0] in R.variable_node(l[1]).parents and l[1] in R.variable_node(l[2]).parents) or (l[2] in R.variable_node(l[1]).parents and l[1] in R.variable_node(l[0]).parents)

def efeitoComum(l, E, R): #True if it is
    return l[0] in R.variable_node(l[1]).parents and l[2] in R.variable_node(l[1]).parents
           
def path(lista, X, Y, R, t):
    if X == Y:
        t.append(X)
        lista.append(t)
        return
    
    if X in t:
        return 
    
    children = R.variable_node(X).children 
    if len(children) > 0:
        for child in children:
            l = []
            for x in t:
                l.append(x)
            l.append(X)
            path(lista, child, Y , R, l)
    parents = R.variable_node(X).parents
    if len(parents) > 1:
        for parent in parents:
            if len(t) > 0 and parent != t[-1]:
                l = []
                for x in t:
                    l.append(x)
                l.append(X)
                path(lista, parent, Y, R, l)
    
T, F = True, False

burglary = BayesNet([
    ('A', '', 0.001),
    ('B', 'A', {T: 0.90, F: 0.05}),
    ('D', 'A', {T: 0.90, F: 0.05}),
    ('E', 'B D', {(T, T): 0.95, (T, F): 0.94, (F, T): 0.29, (F, F): 0.001}),
    ('C', 'B', {T: 0.70, F: 0.01})
])

cond_indep('A', 'B', [], burglary)

False

In [161]:
def cond_indep_explica(X, Y, E, R):
    l = []
    path(l, X, Y, R, [])
    if len(l) == 0:
        return False
    for p in l:
        b = state_e(p, E, R)
        if b[0]:
            return (False, b[1] + " logo " + str(X) + " e " + str(Y) + " não são condicionalmente independentes")
        
    return (True, "Todos os caminhos entre " + str(X) + " e " + str(Y) + " são inativos logo eles estão D-separados por " + str(E) +  ", isto é, são condicionalmente independentes")

def state_e(p, E, R):
    l = p[:3]
    t = p[1:]
    if len(l) < 3:
        return (True, "Não existe um triplo")
    i = inactive_e(l, E, R)
    if i[0]: #True if inactive, False otherwise
        return (False, i[1])
    elif len(t) >= 3:
        return state_e(t, E, R)
    else:
        return (True, "Todos os triplos do caminho " + str(p) + " são ativos")
    
def inactive_e(l, E, R):
    if efeitoCausal(l, E, R):
        if l[1] in E:
            return (True, "Teste")
        else:
            return (False, "Teste")
    elif efeitoComum(l, E, R):
        if l[1] in E:
            return (False, "Teste")
        for node in R.variable_node(l[1]).children:
            if node in E: # check if any descendants of middle cenas is in evidence
                return (False, "Teste")
        return (True, "T")
    else:
        if l[1] in E:
            return (True, "Teste")
        else:
            return (False, "Teste")

cond_indep_explica('A', 'C', ['E'], burglary)

(False,
 "Todos os triplos do caminho ['A', 'B', 'C'] são ativos logo A e C não são condicionalmente independentes")

In [162]:
#que recebe como argumento uma instância  𝑅  de uma Rede de Bayes 
#e que devolve uma lista com todos os triplos distintos  (𝑋,𝑌,𝐸)  tais que se verifica que  
#𝑋  é condicionalmente independente de  𝑌  dado  𝐸 a rede n  𝑅 .
def todos_cond_indeps(R):
    nodes = []
    ret = []
    for node in R.nodes:
        nodes.append(node.variable)
    E = evi(nodes)
    for nodeX in nodes:
        for nodeY in nodes:
            for ev in E:
                if cond_indep(nodeX, nodeY, ev, R):
                    ret.append((nodeX, nodeY, ev))  
    return ret
def evi(nodes):
    ret = []
    for L in range(0, len(nodes)+1):
        for subset in itertools.combinations(nodes, L):
            ret.append(list(subset))
    return ret

def isParent(nodeX, nodeY, bayesNet): 
    return nodeY in bayesNet.variable_node(nodeX).parents

todos_cond_indeps(burglary)

[('A', 'E', ['B', 'D']),
 ('A', 'E', ['A', 'B', 'D']),
 ('A', 'E', ['B', 'D', 'E']),
 ('A', 'E', ['B', 'D', 'C']),
 ('A', 'E', ['A', 'B', 'D', 'E']),
 ('A', 'E', ['A', 'B', 'D', 'C']),
 ('A', 'E', ['B', 'D', 'E', 'C']),
 ('A', 'E', ['A', 'B', 'D', 'E', 'C']),
 ('A', 'C', ['B']),
 ('A', 'C', ['A', 'B']),
 ('A', 'C', ['B', 'D']),
 ('A', 'C', ['B', 'E']),
 ('A', 'C', ['B', 'C']),
 ('A', 'C', ['A', 'B', 'D']),
 ('A', 'C', ['A', 'B', 'E']),
 ('A', 'C', ['A', 'B', 'C']),
 ('A', 'C', ['B', 'D', 'E']),
 ('A', 'C', ['B', 'D', 'C']),
 ('A', 'C', ['B', 'E', 'C']),
 ('A', 'C', ['A', 'B', 'D', 'E']),
 ('A', 'C', ['A', 'B', 'D', 'C']),
 ('A', 'C', ['A', 'B', 'E', 'C']),
 ('A', 'C', ['B', 'D', 'E', 'C']),
 ('A', 'C', ['A', 'B', 'D', 'E', 'C']),
 ('D', 'C', []),
 ('D', 'C', ['A']),
 ('D', 'C', ['B']),
 ('D', 'C', ['D']),
 ('D', 'C', ['E']),
 ('D', 'C', ['C']),
 ('D', 'C', ['A', 'B']),
 ('D', 'C', ['A', 'D']),
 ('D', 'C', ['A', 'E']),
 ('D', 'C', ['A', 'C']),
 ('D', 'C', ['B', 'D']),
 ('D', 'C', ['B', 